In [1]:
import langchain
import langgraph
import openai

print("Environment is ready!")

Environment is ready!


In [21]:
# Sample data for Primary Care Department
primary_care_data = {
    "How can I schedule an appointment with my primary care physician?": "You can schedule online via the patient portal or call our front desk at (312) 555-1234.",
    "Do I need a referral to see a specialist?": "Yes, in most cases your primary care provider must provide a referral.",
    "Can my primary care doctor refill my prescription?": "Yes, prescription refills can be requested through the patient portal or by phone.",
    "What conditions does primary care treat?": "Common conditions include colds, flu, diabetes, hypertension, and preventative care.",
    "Are annual physicals covered by insurance?": "Most insurance plans cover annual physicals at no additional cost.",
    "Can I get blood work done during my primary care visit?": "Yes, most clinics can collect blood samples during your visit.",
    "What should I bring to my first primary care appointment?": "Please bring a photo ID, insurance card, and a list of current medications.",
    "Can primary care doctors diagnose mental health issues?": "Yes, they can diagnose and offer initial treatment or refer to a specialist.",
    "How do I transfer my medical records to this clinic?": "Contact your previous provider and ask them to fax or send records securely.",
    "Do primary care doctors treat children?": "Some do, but we typically refer pediatric patients to the Pediatrics department."
}

In [3]:
# Sample data for ER Department
er_data = {
    "What are the visiting hours for the ER?": "The ER is open 24/7. Visitors may be limited based on patient condition and time of day.",
    "Can I bring someone with me to the ER?": "Yes, one visitor is generally allowed, but check current hospital policy.",
    "How long is the average ER wait time?": "Wait times vary by case urgency, but average is 30-60 minutes.",
    "Do I need an appointment for the ER?": "No appointment is needed for emergency care.",
    "What should I bring to the ER?": "Bring an ID, insurance card, and list of medications.",
    "Are ER doctors available at night?": "Yes, ER staff and doctors are available 24/7.",
    "Do ER visits need insurance?": "No, but having insurance may reduce your out-of-pocket cost.",
    "Can I call ahead before going to the ER?": "You can, but it’s not required. In emergencies, call 911.",
    "Will I be seen faster if I arrive by ambulance?": "Only if it’s a critical emergency. All patients are triaged.",
    "Is the ER equipped to handle pediatric cases?": "Yes, the ER is staffed for adult and pediatric emergencies."
}

In [5]:
# Sample data for Radiology Department
radiology_data = {
    "How should I prepare for a CT scan?": "Avoid eating for 4 hours prior. Wear loose clothing and inform us if you're pregnant.",
    "Can I eat before an MRI?": "Yes, unless instructed otherwise. Certain contrast scans may require fasting.",
    "Is an X-ray safe during pregnancy?": "Generally avoided unless medically necessary. Always inform the technician.",
    "When will I receive my imaging results?": "Results are typically available within 24-48 hours through your physician.",
    "Do I need a referral for an MRI?": "Yes, most insurance plans require a referral from a doctor.",
    "What is the difference between a CT scan and an MRI?": "CT uses X-rays for detailed images; MRI uses magnets for soft tissues.",
    "How long does an ultrasound take?": "Most ultrasound exams take between 30 to 60 minutes.",
    "Will I need to remove jewelry for an X-ray?": "Yes, to avoid interference with the image.",
    "Are radiology services covered by insurance?": "Most diagnostic imaging is covered but may require pre-authorization.",
    "Can I get a copy of my scan?": "Yes, you can request digital or printed copies through the radiology department."
}

In [7]:
# Sample data for Pediatrics Department
pediatrics_data = {
    "What vaccines are required for a 2-year-old?": "DTaP, Hib, Polio, and MMR are commonly recommended.",
    "My child has a fever; when should I worry?": "Call if it’s over 100.4°F for infants or lasts more than 3 days.",
    "Is a cough normal after a cold?": "Yes, it may persist for 1-2 weeks, but monitor for wheezing.",
    "Can teething cause diarrhea?": "Mild loose stools can occur, but persistent diarrhea warrants a visit.",
    "What foods should I avoid introducing to a 6-month-old?": "Avoid honey and choking hazards like nuts.",
    "How often should my child see a pediatrician?": "Regular checkups are recommended at key milestones, typically every few months in the first year.",
    "Can I bring both my children to the same appointment?": "Yes, but please notify us in advance for scheduling purposes.",
    "Are virtual visits available for children?": "Yes, virtual visits are available for certain non-emergency concerns.",
    "What should I do if my child missed a vaccine?": "Call your pediatrician to create a catch-up immunization schedule.",
    "Is my child too young for the flu shot?": "Children 6 months and older can receive the flu shot."
}

In [9]:
# Sample data for Cardiology Department
cardiology_data = {
    "How do I schedule a cardiology appointment?": "Call (312) 555-4567 or request an appointment through the patient portal.",
    "What are signs I should see a cardiologist?": "Shortness of breath, chest pain, palpitations, or high blood pressure are key signs.",
    "Can cardiologists perform EKGs during visits?": "Yes, EKGs are commonly done during routine cardiology appointments.",
    "Do I need a referral to see a cardiologist?": "Most insurance plans require a referral from your primary care provider.",
    "What should I bring to a cardiology appointment?": "Bring ID, insurance card, a list of medications, and previous test results if available.",
    "How do I prepare for a stress test?": "Wear comfortable clothes and avoid caffeine or heavy meals beforehand.",
    "Are virtual cardiology visits available?": "Yes, for follow-up appointments and consultations, virtual visits are offered.",
    "What does a normal blood pressure reading look like?": "A typical healthy range is around 120/80 mmHg.",
    "Can a cardiologist help with high cholesterol?": "Yes, they can assess your risks and create a treatment plan.",
    "Are heart palpitations always serious?": "Not always, but they should be evaluated if persistent or accompanied by other symptoms."
}

In [11]:
# Sample data for Billing & Insurance Department
billing_insurance_data = {
    "Can you explain my recent medical bill?": "Yes, please provide your billing account number and we’ll go over the charges.",
    "Does Northwestern accept my insurance plan?": "We accept most major insurance plans. Check our website or call to verify your provider.",
    "How can I pay my bill online?": "Visit the patient portal, log in, and go to the 'Billing' section to pay securely.",
    "Why did I receive multiple bills for one visit?": "You may receive separate bills from the hospital and individual specialists.",
    "Can I set up a payment plan?": "Yes, payment plans are available—contact billing services at (312) 555-7890.",
    "What is an EOB and how is it different from a bill?": "An Explanation of Benefits (EOB) is not a bill. It shows what your insurance covered.",
    "What if my insurance denied a claim?": "You can appeal with your insurance provider or contact our office for support.",
    "Are telehealth visits covered by insurance?": "Many insurers cover telehealth, but check with your provider for details.",
    "Do I need to pay a copay for every visit?": "Usually yes, but the amount depends on your insurance plan and visit type.",
    "Can I get a detailed receipt for my HSA or FSA?": "Yes, detailed receipts are available upon request for reimbursement purposes."
}

In [13]:
import difflib

# === Generic retrieval function for department Q&A dictionaries ===
def get_best_match_answer(user_question: str, qa_dict: dict) -> str:
    """
    Given a user's question and a dictionary of Q&A pairs,
    this function finds the closest matching question using fuzzy matching
    and returns the corresponding answer.
    """
    # Extract list of known questions
    questions = list(qa_dict.keys())
    
    # Find the closest matching question using difflib
    best_match = difflib.get_close_matches(user_question, questions, n=1, cutoff=0.4)

    if best_match:
        # Return the answer for the best-matching question
        return qa_dict[best_match[0]]
    else:
        # Default response if no good match is found
        return "I'm sorry, I couldn’t find an exact answer to your question. Please contact the department directly."

# === Example usage ===
sample_question = "How do I pay my bill online?"
answer = get_best_match_answer(sample_question, billing_insurance_data)
print("Answer:", answer)

Answer: Visit the patient portal, log in, and go to the 'Billing' section to pay securely.


In [15]:
import os
from dotenv import load_dotenv
import openai

# Load environment variables from .env file
load_dotenv()

# Set the OpenAI key from environment
openai.api_key = os.getenv("OPENAI_API_KEY")

# Confirm it's working
if openai.api_key:
    print("OpenAI key loaded successfully.")
else:
    print("OpenAI key not found. Check your .env file.")

OpenAI key loaded successfully.


In [23]:
# === Imports ===
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage

# === State passed between nodes ===
class HospitalState(dict):
    input: str
    intents: list
    response: str

# === Operator node that classifies intent using OpenAI ===
def operator_node(state: HospitalState) -> HospitalState:
    message = state['input']
    intents = classify_intent(message)  # already implemented earlier
    return {**state, "intents": intents}

operator_runnable = RunnableLambda(operator_node)

# === Department responder generator ===
def make_department_node(name, qa_dict):
    def handler(state: HospitalState) -> HospitalState:
        message = state['input']
        answer = get_best_match_answer(message, qa_dict)  # already implemented earlier
        return {**state, "response": answer}
    return RunnableLambda(handler)

# === All department agent nodes ===
department_nodes = {
    "er": make_department_node("er", er_data),
    "primary_care": make_department_node("primary_care", primary_care_data),
    "radiology": make_department_node("radiology", radiology_data),
    "pediatrics": make_department_node("pediatrics", pediatrics_data),
    "cardiology": make_department_node("cardiology", cardiology_data),
    "billing_insurance": make_department_node("billing_insurance", billing_insurance_data),
}

In [25]:
# === Step 1: Build the graph ===
builder = StateGraph(HospitalState)

# Add the operator node (intent classification)
builder.add_node("operator", operator_runnable)

# Add department nodes
for dept, node in department_nodes.items():
    builder.add_node(dept, node)

# === Step 2: Define routing logic ===
# This function chooses the next node based on the first detected intent
def route_from_intent(state: HospitalState) -> str:
    intents = state.get("intents", [])
    if not intents:
        return END
    intent = intents[0]  # Only use the first intent for now
    return intent if intent in department_nodes else END

# === Step 3: Define graph edges ===
builder.set_entry_point("operator")
builder.add_conditional_edges("operator", route_from_intent)
for dept in department_nodes:
    builder.add_edge(dept, END)

# === Step 4: Compile the graph ===
graph = builder.compile()

In [29]:
# === Operator Agent: Classify Intent with OpenAI ===
def classify_intent(message: str) -> list:
    """
    Uses OpenAI to classify the user's intent from a list of possible departments.
    Returns a list of relevant department names (as keys used in the routing logic).
    """
    prompt = (
        f"Classify the user's intents based on the following input: '{message}'. "
        "Possible intents include: ER, primary_care, radiology, pediatrics, cardiology, billing_insurance. "
        "Return all relevant intents as a comma-separated list without any duplication. "
        "Ensure that each intent is distinct and only included if it is clearly relevant to the input."
    )

    response = openai.ChatCompletion.create(
        model="gpt-4o",  # You may switch to gpt-3.5-turbo if needed
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    # Parse and clean the response
    raw_intents = response['choices'][0]['message']['content']
    intent_list = [intent.strip().lower() for intent in raw_intents.split(',') if intent.strip()]
    return intent_list

In [33]:
from openai import OpenAI

# Initialize OpenAI client using .env credentials
client = OpenAI()

# === Operator Agent: Classify Intent ===
def classify_intent(message: str) -> list:
    """
    Uses OpenAI's v1+ SDK to classify user intent for hospital departments.
    Returns a list of relevant department keys (e.g., 'pediatrics').
    """
    prompt = (
        f"Classify the user's intents based on the following input: '{message}'. "
        "Possible intents include: ER, primary_care, radiology, pediatrics, cardiology, billing_insurance. "
        "Return all relevant intents as a comma-separated list without any duplication. "
        "Ensure that each intent is distinct and only included if it is clearly relevant to the input."
    )

    response = client.chat.completions.create(
        model="gpt-4o",  # You may switch to 'gpt-4o-mini' if needed
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    # Extract and clean intent list
    content = response.choices[0].message.content
    intents = [intent.strip().lower() for intent in content.split(",") if intent.strip()]
    return intents

In [35]:
# === 6 Test Inquiries from the assignment ===
test_questions = [
    "How can I tell if my child has RSV (Respiratory Syncytial Virus)?",
    "Can I visit my friend in the ER, and are there any restrictions?",
    "How should I prepare for a CT scan, and are there any dietary restrictions?",
    "I want to check if my recent test result has been reviewed by my primary care physician yet.",
    "Do you have any available appointments with a cardiologist next week?",
    "Can you help me understand my recent medical bill and whether my insurance covered the costs?"
]

# === Run each question through the LangGraph ===
for i, question in enumerate(test_questions, start=1):
    print(f"\n--- Test Case {i} ---")
    print("User:", question)

    # Run the graph
    result = graph.invoke({"input": question})

    # Show the result
    print("AI:", result.get("response"))


--- Test Case 1 ---
User: How can I tell if my child has RSV (Respiratory Syncytial Virus)?
AI: Regular checkups are recommended at key milestones, typically every few months in the first year.

--- Test Case 2 ---
User: Can I visit my friend in the ER, and are there any restrictions?
AI: Yes, the ER is staffed for adult and pediatric emergencies.

--- Test Case 3 ---
User: How should I prepare for a CT scan, and are there any dietary restrictions?
AI: Avoid eating for 4 hours prior. Wear loose clothing and inform us if you're pregnant.

--- Test Case 4 ---
User: I want to check if my recent test result has been reviewed by my primary care physician yet.
AI: You can schedule online via the patient portal or call our front desk at (312) 555-1234.

--- Test Case 5 ---
User: Do you have any available appointments with a cardiologist next week?
AI: Most insurance plans require a referral from your primary care provider.

--- Test Case 6 ---
User: Can you help me understand my recent medic